# Explore the Transformed Play by Play Data

## Setup - Paths - Dependencies

In [25]:
from config import recent_play_by_play, recent_clean_db

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load Play by Play data
pbp_raw_df = pd.read_csv(recent_play_by_play, low_memory=False)

# pbp_raw_df.head()


In [26]:
# # Rename to df for convenience
# df = pbp_raw_df

# # Filter for relevant events: Faceoffs and Goals
# faceoff_events = df[df['Event_type'] == 'Faceoff']
# goal_events = df[df['Event_type'] == 'Goal']

# # Merge faceoff events with subsequent goal events within the same game and period
# merged_df = pd.merge(
#     faceoff_events[['Game_ID', 'Period', 'Time', 'Primary_team']],
#     goal_events[['Game_ID', 'Period', 'Time', 'Primary_team']],
#     on=['Game_ID', 'Period'],
#     suffixes=('_faceoff', '_goal')
# )

# # Calculate the time difference between faceoff and goal
# merged_df['time_diff'] = merged_df['Time_goal'] - merged_df['Time_faceoff']

# # Filter only instances where the goal happens after the faceoff
# merged_df = merged_df[merged_df['time_diff'] > 0]


# # Count how often a goal is scored within 10 seconds and 5 seconds of a faceoff
# goals_within_10s = (merged_df['time_diff'] <= 10).sum()
# goals_within_5s = (merged_df['time_diff'] <= 5).sum()

# # Count the number of times each team has scored a goal within 5 seconds of a faceoff
# teams_scoring_within_5s = merged_df[merged_df['time_diff'] <= 5]['Primary_team_goal'].value_counts()

# # Display results
# # goals_within_10s, goals_within_5s, teams_scoring_within_5s


## Data Elporation

### Faceoff Danger
- Table of how many times a team has scored within 3,5, 7, and 10 seconds of a faceoff

In [27]:
# Recalculate the time difference properly
merged_df = pd.merge(
    faceoff_events[['Game_ID', 'Period', 'Time', 'Primary_team']],
    goal_events[['Game_ID', 'Period', 'Time', 'Primary_team']],
    on=['Game_ID', 'Period'],
    suffixes=('_faceoff', '_goal')
)

# Ensure that the goal occurs after the faceoff in time
merged_df = merged_df[merged_df['Time_goal'] > merged_df['Time_faceoff']]

# Calculate the correct time difference between faceoff and goal
merged_df['time_diff'] = merged_df['Time_goal'] - merged_df['Time_faceoff']

# Count goals within specific time frames
goals_within_10s = (merged_df['time_diff'] <= 10).sum()
goals_within_5s = (merged_df['time_diff'] <= 5).sum()

# Count goals per team in different time frames
teams_goal_counts = merged_df.groupby('Primary_team_goal')['time_diff'].agg(
    within_3s=lambda x: (x <= 3).sum(),
    within_5s=lambda x: (x <= 5).sum(),
    within_7s=lambda x: (x <= 7).sum(),
    within_10s=lambda x: (x <= 10).sum()
).reset_index()

# Add the total goals scored by each team
total_goals_per_team = goal_events['Primary_team'].value_counts().reset_index()
total_goals_per_team.columns = ['Primary_team_goal', 'Total_Goals']

# Merge with the team goal counts
teams_goal_counts = teams_goal_counts.merge(total_goals_per_team, on='Primary_team_goal', how='left')



In [28]:
teams_goal_counts.head()

,Primary_team_goal,within_3s,within_5s,within_7s,within_10s,Total_Goals
0,Air Force,0,2,4,8,55
1,Alaska,1,4,6,9,54
2,Alaska Anchorage,0,0,1,2,55
3,American Intl,0,1,1,4,57
4,Arizona State,0,1,1,2,90
